<a href="https://colab.research.google.com/github/Tunji17/huggingface-tasks/blob/main/language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.48.3


# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [ ]:
from datasets import load_dataset, DatasetDict
datasets = load_dataset('2nji/Shakespeare_Corpus')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
# first_split = shakespeare.train_test_split(test_size=0.2, seed=42)
# train_dataset = first_split["train"]
# test_dataset = first_split["test"]

In [ ]:
# second_split = train_dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = second_split["train"]
# val_dataset = second_split["test"]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets['train'])

,text
0,RATCLIFF: Your highness told me I should post before.
1,QUEEN ELIZABETH: That at her hands which the king's King forbids.
2,BENVOLIO: It was. What sadness lengthens Romeo's hours?
3,"GLOUCESTER: The selfsame name, but one of better nature."
4,"First Citizen: Our business is not unknown to the senate; they have had inkling this fortnight what we intend to do, which now we'll show 'em in deeds. They say poor suitors have strong breaths: they shall know we have strong arms too."
5,"LADY CAPULET: That same villain, Romeo."
6,"MENENIUS: Consider further, That when he speaks not like a citizen, You find him like a soldier: do not take His rougher accents for malicious sounds, But, as I say, such as become a soldier, Rather than envy you."
7,"ARIEL: All hail, great master! grave sir, hail! I come To answer thy best pleasure; be't to fly, To swim, to dive into the fire, to ride On the curl'd clouds, to thy strong bidding task Ariel and all his quality."
8,"CORIOLANUS: If, Tullus, Not yet thou knowest me, and, seeing me, dost not Think me for the man I am, necessity Commands me name myself."
9,GLOUCESTER:


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [ ]:
model_checkpoint = "EleutherAI/gpt-neo-125m"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
# datasets = DatasetDict({"train": train_dataset, "test": test_dataset, "validation": val_dataset})

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4621 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1445 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1156 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
print("Train shape", tokenized_datasets["train"].shape)
print("Test shape", tokenized_datasets["test"].shape)
print("Validation shape", tokenized_datasets["validation"].shape)

Train shape (4621, 2)
Test shape (1445, 2)
Validation shape (1156, 2)


Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
# block_size = tokenizer.model_max_length
# print(f"Block size: {block_size}")

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4621 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1445 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1156 [00:00<?, ? examples/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][2]["input_ids"])

' caps and legs: you wear out a good wholesome forenoon in hearing a cause between an orange wife and a fosset-seller; and then rejourn the controversy of three pence to a second day of audience. When you are hearing a matter between party and party, if you chance to be pinched with the colic, you make faces like mummers; set up the bloody flag against all patience; and, in roaring for a chamber-pot, dismiss the controversy bleeding the more entangled by your hearing: all the peace you make in their cause is, calling both the parties knaves. You are a pair of'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]

print(model_name)
training_args = TrainingArguments(
    f"{model_name}-finetuned-shakespeare",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

gpt-neo-125m


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2nji (_2nji) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,4.184366
2,No log,4.124846
3,4.089200,4.112580


TrainOutput(global_step=555, training_loss=4.070328941001549, metrics={'train_runtime': 221.0944, 'train_samples_per_second': 20.082, 'train_steps_per_second': 2.51, 'total_flos': 289939878051840.0, 'train_loss': 4.070328941001549, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 61.10


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1740651373.bb3b7466655b.2938.0:   0%|          | 0.00/6.92k [00:00<?, ?B/s]

events.out.tfevents.1740651767.bb3b7466655b.2938.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/2nji/gpt-neo-125m-finetuned-shakespeare/commit/8e2fbb83db027c21f4bacd25c1549d31b231c84b', commit_message='End of training', commit_description='', oid='8e2fbb83db027c21f4bacd25c1549d31b231c84b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/2nji/gpt-neo-125m-finetuned-shakespeare', endpoint='https://huggingface.co', repo_type='model', repo_id='2nji/gpt-neo-125m-finetuned-shakespeare'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("2nji/gpt-neo-125m-finetuned-shakespeare")

README.md:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/2nji/gpt-neo-125m-finetuned-shakespeare/commit/2a82fe4a7eb33c112432c1af4c70bb3c22e71c9a', commit_message='Upload tokenizer', commit_description='', oid='2a82fe4a7eb33c112432c1af4c70bb3c22e71c9a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/2nji/gpt-neo-125m-finetuned-shakespeare', endpoint='https://huggingface.co', repo_type='model', repo_id='2nji/gpt-neo-125m-finetuned-shakespeare'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='2nji/gpt-neo-125m-finetuned-shakespeare')

generator("And all that", do_sample=True, min_length=20)

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "answerdotai/ModernBERT-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/4621 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1445 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1156 [00:00<?, ? examples/s]

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4621 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1445 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1156 [00:00<?, ? examples/s]

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-mask-finetuned-shakespeare",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2nji (_2nji) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,2.312828
2,No log,2.214971
3,2.300200,2.239456


TrainOutput(global_step=591, training_loss=2.2752910685014807, metrics={'train_runtime': 297.5445, 'train_samples_per_second': 15.83, 'train_steps_per_second': 1.986, 'total_flos': 401418995466240.0, 'train_loss': 2.2752910685014807, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 9.34


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

events.out.tfevents.1740994020.dee571fcdb99.605.0:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

events.out.tfevents.1740994333.dee571fcdb99.605.1:   0%|          | 0.00/630 [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/2nji/ModernBERT-base-mask-finetuned-shakespeare/commit/482a4a85503e66f3e3b62ced37974a47c49f6535', commit_message='End of training', commit_description='', oid='482a4a85503e66f3e3b62ced37974a47c49f6535', pr_url=None, repo_url=RepoUrl('https://huggingface.co/2nji/ModernBERT-base-mask-finetuned-shakespeare', endpoint='https://huggingface.co', repo_type='model', repo_id='2nji/ModernBERT-base-mask-finetuned-shakespeare'), pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("2nji/ModernBERT-base-mask-finetuned-shakespeare")

README.md:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/2nji/ModernBERT-base-mask-finetuned-shakespeare/commit/504f6ddec6476cf1cb858008fd547c3f2aec87c5', commit_message='Upload tokenizer', commit_description='', oid='504f6ddec6476cf1cb858008fd547c3f2aec87c5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/2nji/ModernBERT-base-mask-finetuned-shakespeare', endpoint='https://huggingface.co', repo_type='model', repo_id='2nji/ModernBERT-base-mask-finetuned-shakespeare'), pr_revision=None, pr_num=None)

In [ ]:
import torch
from transformers import pipeline
from pprint import pprint

pipe = pipeline(
    "fill-mask",
    model="2nji/ModernBERT-base-mask-finetuned-shakespeare",
    torch_dtype=torch.bfloat16,
)

input_text = "Thou [MASK] on [MASK]."
results = pipe(input_text)
pprint(results)



Device set to use cuda:0


[[{'score': 0.71875,
   'sequence': '[CLS]Thou art on[MASK].[SEP]',
   'token': 1445,
   'token_str': ' art'},
  {'score': 0.1416015625,
   'sequence': '[CLS]Thou hast on[MASK].[SEP]',
   'token': 16579,
   'token_str': ' hast'},
  {'score': 0.014892578125,
   'sequence': '[CLS]Thou be on[MASK].[SEP]',
   'token': 320,
   'token_str': ' be'},
  {'score': 0.00701904296875,
   'sequence': '[CLS]Thou Art on[MASK].[SEP]',
   'token': 3975,
   'token_str': ' Art'},
  {'score': 0.0042724609375,
   'sequence': '[CLS]Thou call on[MASK].[SEP]',
   'token': 1067,
   'token_str': ' call'}],
 [{'score': 0.1767578125,
   'sequence': "[CLS]Thou[MASK] on't.[SEP]",
   'token': 626,
   'token_str': "'t"},
  {'score': 0.146484375,
   'sequence': '[CLS]Thou[MASK] on me.[SEP]',
   'token': 479,
   'token_str': ' me'},
  {'score': 0.0419921875,
   'sequence': '[CLS]Thou[MASK] on it.[SEP]',
   'token': 352,
   'token_str': ' it'},
  {'score': 0.0419921875,
   'sequence': '[CLS]Thou[MASK] on earth.[SEP]',
  